In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `forme` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `forme`


In [4]:
from PIL import Image
import requests
from transformers import SiglipProcessor, SiglipModel, SiglipVisionModel, SiglipImageProcessor,AutoModel,AutoImageProcessor,SiglipConfig, Gemma3ImageProcessor
import torch
import inspect
import matplotlib.pyplot as plt
from peft import PeftModel
from transformers import AutoProcessor, AutoModel


In [5]:
url = "/content/drive/MyDrive/images/0.jpg"
image = Image.open(url)
resize = image.resize((896, 896))
width, height = resize.size

In [6]:
# Load model and processor
processor = SiglipProcessor.from_pretrained("google/siglip-base-patch16-224")
model = SiglipModel.from_pretrained("google/siglip-base-patch16-224")

url = "/content/drive/MyDrive/images/0.jpg"

# Your image and text
image = Image.open(url).convert("RGB")
text = "a description of the image"
inputs = processor(images=image, text=text, return_tensors="pt")

outputs = model(**inputs)
if outputs != None:
  print("+")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/711 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/813M [00:00<?, ?B/s]

+


In [7]:
processor = SiglipImageProcessor.from_pretrained("google/siglip-base-patch16-224")
model = SiglipVisionModel.from_pretrained("google/siglip-base-patch16-224")

image = Image.open(url).convert("RGB")
inputs = processor(images=image, return_tensors="pt")

outputs = model(**inputs)
if outputs != None:
  print("+")

+


In [8]:
_VISION_CONFIG = {
    "hidden_size": 1152,
    "intermediate_size": 4304,
    "num_hidden_layers": 27,
    "num_attention_heads": 16,
    "num_channels": 3,
    "image_size": 896,
    "patch_size": 14,
    "hidden_act": "gelu_pytorch_tanh",
    "layer_norm_eps": 1e-6,
    "attention_dropout": 0.0,
    "vision_use_head": False,
}
vision_config=SiglipConfig(**_VISION_CONFIG)

In [9]:
#model = AutoModel.from_config(config=vision_config)
model = SiglipModel(vision_config).vision_model.eval()
print(model)

SiglipVisionTransformer(
  (embeddings): SiglipVisionEmbeddings(
    (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), padding=valid)
    (position_embedding): Embedding(196, 768)
  )
  (encoder): SiglipEncoder(
    (layers): ModuleList(
      (0-11): 12 x SiglipEncoderLayer(
        (layer_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attn): SiglipAttention(
          (k_proj): Linear(in_features=768, out_features=768, bias=True)
          (v_proj): Linear(in_features=768, out_features=768, bias=True)
          (q_proj): Linear(in_features=768, out_features=768, bias=True)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (layer_norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): SiglipMLP(
          (activation_fn): PytorchGELUTanh()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bi

In [10]:
model_id = "google/gemma-3-4b-it"

processor1 = AutoProcessor.from_pretrained(model_id)

members1 = dict(inspect.getmembers(processor1))

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [11]:
print(members1.keys())

dict_keys(['__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_auto_class', '_create_repo', '_get_arguments_from_pretrained', '_get_files_timestamps', '_merge_kwargs', '_process_messages_for_chat_template', '_upload_modified_files', 'apply_chat_template', 'attributes', 'batch_decode', 'boi_token', 'chat_template', 'decode', 'feature_extractor_class', 'from_args_and_dict', 'from_pretrained', 'full_image_sequence', 'get_possibly_dynamic_module', 'get_processor_dict', 'image_processor', 'image_processor_class', 'image_seq_length', 'image_token', 'image_token_id', 'model_input_names', 'optional_attributes', 'optional_call_args', 'post_process_image_text_t

In [12]:
import inspect

for name, member in members1.items():

    if inspect.ismethod(member):
        kind = "method"
    elif inspect.isfunction(member):
        kind = "function"
    elif inspect.isbuiltin(member):
        kind = "builtin function"
    elif inspect.isclass(member):
        kind = "class"
    elif isinstance(member, property):
        kind = "property"
    elif inspect.ismodule(member):
        kind = "module"
    elif inspect.isroutine(member):
        kind = "routine"
    elif inspect.isdatadescriptor(member):
        kind = "data descriptor"
    else:
        kind = "attribute / data"

    if name == "image_processor":
      print(f"{name}: {kind} = {repr(member)}")

image_processor: attribute / data = Gemma3ImageProcessor {
  "do_convert_rgb": null,
  "do_normalize": true,
  "do_pan_and_scan": null,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "Gemma3ImageProcessor",
  "image_seq_length": 256,
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "pan_and_scan_max_num_crops": null,
  "pan_and_scan_min_crop_size": null,
  "pan_and_scan_min_ratio_to_activate": null,
  "processor_class": "Gemma3Processor",
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 896,
    "width": 896
  }
}



In [13]:
image_processor = processor1.image_processor
print(image_processor)

Gemma3ImageProcessor {
  "do_convert_rgb": null,
  "do_normalize": true,
  "do_pan_and_scan": null,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "Gemma3ImageProcessor",
  "image_seq_length": 256,
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "pan_and_scan_max_num_crops": null,
  "pan_and_scan_min_crop_size": null,
  "pan_and_scan_min_ratio_to_activate": null,
  "processor_class": "Gemma3Processor",
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 896,
    "width": 896
  }
}



In [14]:
#inputs = image_processor(images=image, return_tensors="pt")
inputs = image_processor(images=image, return_tensors="pt", size={"height": 224, "width": 224})
# Remove unsupported keys before calling the model
if 'num_crops' in inputs:
    del inputs['num_crops']
print(inputs)

{'pixel_values': tensor([[[[-0.5765, -0.6941, -0.8196,  ..., -0.6941, -0.7020, -0.7176],
          [-0.6627, -0.7255, -0.6941,  ..., -0.6627, -0.6863, -0.7098],
          [-0.7882, -0.7804, -0.5529,  ..., -0.6627, -0.6863, -0.7098],
          ...,
          [ 0.3412, -0.0353, -0.0745,  ..., -0.3412, -0.4667, -0.5294],
          [ 0.2863, -0.0196, -0.1216,  ...,  0.0275, -0.4431, -0.5216],
          [ 0.0980,  0.0980,  0.2078,  ...,  0.1765, -0.4510, -0.5843]],

         [[-0.5294, -0.6627, -0.7961,  ..., -0.6863, -0.6863, -0.6863],
          [-0.6157, -0.6941, -0.6706,  ..., -0.6549, -0.6627, -0.6784],
          [-0.7412, -0.7490, -0.5294,  ..., -0.6549, -0.6706, -0.6784],
          ...,
          [ 0.3961,  0.0118, -0.0431,  ..., -0.2471, -0.3804, -0.4667],
          [ 0.3490,  0.0353, -0.0902,  ...,  0.1294, -0.3569, -0.4588],
          [ 0.1608,  0.1451,  0.2471,  ...,  0.2784, -0.3569, -0.5137]],

         [[-0.4039, -0.5843, -0.7490,  ..., -0.6471, -0.6549, -0.6627],
          [-0

In [16]:
#outputs = model(**inputs)
outputs = model(pixel_values=inputs["pixel_values"])
print(outputs.keys())
print(outputs['last_hidden_state'].size())
print(outputs['pooler_output'].size())

odict_keys(['last_hidden_state', 'pooler_output'])
torch.Size([1, 196, 768])
torch.Size([1, 768])


In [1]:
!pip install peft transformers accelerate bitsandbytes

In [2]:
from transformers import AutoModel, AutoProcessor
from peft import get_peft_model, LoraConfig, TaskType

# Load base Gemma 3 vision model (ensure vision_config is passed correctly)
model = AutoModel.from_pretrained("google/gemma-3-vision-tower")
processor = AutoProcessor.from_pretrained("google/gemma-3-vision-tower")

# Freeze base model
for param in model.parameters():
    param.requires_grad = False


OSError: google/gemma-3-vision-tower is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
lora_config = LoraConfig(
    r=16,                                # rank of adaptation
    lora_alpha=32,                       # scaling factor
    target_modules=["q_proj", "v_proj"], # depends on model architecture
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.FEATURE_EXTRACTION  # or TaskType.IMAGE_CLASSIFICATION
)
lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()